In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv('dataset.csv')

# Display the first few rows of the dataset
print(df.head())


  Patient ID         Day  Resting heart rate  Active heart rate   HRV  \
0       P001  01/01/2024                  78                 90  44.7   
1       P001  02/01/2024                  74                127  63.4   
2       P001  03/01/2024                  76                111  42.5   
3       P001  04/01/2024                  71                111  31.0   
4       P001  05/01/2024                  61                113  37.2   

   Sleep duration  Respiratory rate  Body temperature  \
0             6.6              12.4              37.2   
1             6.3              19.8              36.9   
2             7.0              19.1              36.5   
3             8.4              15.1              37.1   
4             6.3              18.1              37.3   

   Blood oxygen saturation  Activity levels  ...  Fetal movement (kick count)  \
0                       95             2084  ...                            0   
1                       97             7041  ...        

In [5]:
# Convert 'Day' column to datetime
df['Day'] = pd.to_datetime(df['Day'], format='%d/%m/%Y')

# Sort by Patient ID and Day to ensure chronological order
df = df.sort_values(by=['Patient ID', 'Day'])

# Convert continuous columns to numeric if necessary
continuous_columns = ['Resting heart rate', 'Active heart rate', 'HRV', 'Sleep duration', 
                      'Respiratory rate', 'Body temperature', 'Blood oxygen saturation', 
                      'Activity levels', 'Calories burnt', 'Weight gain (kg)']
df[continuous_columns] = df[continuous_columns].apply(pd.to_numeric, errors='coerce')


In [6]:
import numpy as np

# Function to create time series windows (e.g., 10 days of data per patient)
def create_time_series_windows(df, time_steps):
    data = []
    for patient_id in df['Patient ID'].unique():
        patient_data = df[df['Patient ID'] == patient_id][continuous_columns].values
        for i in range(len(patient_data) - time_steps):
            data.append(patient_data[i:i + time_steps])
    return np.array(data)

time_steps = 10  # Example: 10 days of data
time_series_data = create_time_series_windows(df, time_steps)
print(time_series_data.shape)  # Should print something like (num_samples, 10, num_features)


(270, 10, 10)


In [7]:
from timegan import TimeGAN

# Initialize the TimeGAN model with the appropriate time steps and feature count
num_features = len(continuous_columns)  # Number of features (e.g., 10 features)
model = TimeGAN(time_steps=time_steps, features=num_features)

# Train the model on the time series data
model.fit(time_series_data)

# Generate synthetic time series data (e.g., 1000 samples)
synthetic_data = model.sample(1000)
print(synthetic_data.shape)  # Should print (1000, time_steps, num_features)


ModuleNotFoundError: No module named 'timegan'

In [ ]:
# Convert the synthetic data into a DataFrame for easy manipulation
synthetic_df = pd.DataFrame(synthetic_data.reshape(-1, num_features), columns=continuous_columns)

# Save the synthetic data to a new CSV file
synthetic_df.to_csv('synthetic_pregnancy_data.csv', index=False)
